In [215]:
import pandas as pd

# Step 1: Base URL for the JSON files
base_url = 'https://github.com/SunilGorantla/QT_Batch_0011/raw/main/Machine_Learning/datasets/chess/train'

# Step 2: List of JSON filenames to be read
file_names = [
    'tournament_1.json', 'tournament_1.json', 'tournament_10.json', 'tournament_10.json', 'tournament_100.json', 'tournament_100.json', 
    'tournament_101.json', 'tournament_101.json', 'tournament_102.json', 'tournament_102.json', 'tournament_103.json', 'tournament_103.json',
]

# Step 3: Initialize an empty list to store DataFrames
dfs = []

# Step 4: Loop through the file names, read each file, and append to the list
for file_name in file_names:
    file_url = f"{base_url}/{file_name}"
    try:
        df = pd.read_json(file_url)
        dfs.append(df)
    except Exception as e:
        print(f"Error reading {file_url}: {e}")

# Step 5: Concatenate all DataFrames into a single DataFrame
combined_df = pd.concat(dfs, ignore_index=True)

# Step 6: Reset index to ensure unique index values
combined_df.reset_index(drop=True, inplace=True)

# Step 7: Check the structure of the DataFrame
print("Columns in the DataFrame:", combined_df.columns)
print("Sample data in the DataFrame:", combined_df.head())

# Step 8: Handle the 'games' column
if 'games' in combined_df.columns:
    # Ensure 'games' column contains lists
    if combined_df['games'].apply(lambda x: isinstance(x, list)).all():
        # Explode the 'games' column to separate rows for each game
        games_df = combined_df.explode('games')
        
        # Reset index after exploding
        games_df.reset_index(drop=True, inplace=True)
        
        # Convert the exploded 'games' column to a DataFrame
        games_details_df = pd.json_normalize(games_df['games'])

        # Concatenate the new games details DataFrame with the original DataFrame (excluding the old 'games' column)
        final_df = pd.concat([games_df.drop(columns='games'), games_details_df], axis=1)

        # Reset index again after concatenation
        final_df.reset_index(drop=True, inplace=True)

        # Display the new DataFrame structure
        print("Final DataFrame Structure:")
        print(final_df.head())

        # Optional: Save the new DataFrame to a CSV file for further analysis
        final_df.to_csv('simplified_chess_data.csv', index=False)
    else:
        print("The 'games' column does not contain lists.")
else:
    print("The 'games' column is missing.")


Columns in the DataFrame: Index(['name', 'start_date', 'end_date', 'games', 'tours', 'time_control'], dtype='object')
Sample data in the DataFrame:            name  start_date    end_date  \
0  tournament_1  2014-01-08  2014-01-17   
1  tournament_1  2014-01-08  2014-01-17   
2  tournament_1  2014-01-08  2014-01-17   
3  tournament_1  2014-01-08  2014-01-17   
4  tournament_1  2014-01-08  2014-01-17   

                                               games  tours time_control  
0  [{'white': '贾叶珍', 'black': '范辰妮', 'date': '201...      9        rapid  
1  [{'white': '曹灵缨', 'black': '陆桂姐', 'date': '201...      9        rapid  
2  [{'white': '刘家坚', 'black': '沈岚鸿', 'date': '201...      9        rapid  
3  [{'white': '沈岚鸿', 'black': '李文子', 'date': '201...      9        rapid  
4  [{'white': '范存妞', 'black': '沈岚鸿', 'date': '201...      9        rapid  
Final DataFrame Structure:
           name  start_date    end_date  tours time_control white black  \
0  tournament_1  2014-01-08  2014-01-17  

In [216]:
final_df = final_df.drop_duplicates()

In [37]:
final_df.head()

,name,start_date,end_date,tours,time_control,white,black,date,result,id
0,tournament_1,2014-01-08,2014-01-17,9,rapid,贾叶珍,范辰妮,2014-01-08,0.5,tournament_1_1
1,tournament_1,2014-01-08,2014-01-17,9,rapid,吕亚光,李嘉爵,2014-01-08,0.5,tournament_1_2
2,tournament_1,2014-01-08,2014-01-17,9,rapid,刘奇喜,刘晓鹏,2014-01-08,0.5,tournament_1_3
3,tournament_1,2014-01-08,2014-01-17,9,rapid,陆桂姐,郑新聪,2014-01-08,1.0,tournament_1_4
4,tournament_1,2014-01-08,2014-01-17,9,rapid,李汶玲,叶天英,2014-01-08,0.5,tournament_1_5


In [219]:

# Create the 'Pattern' column by splitting the 'id' column after the first "_"
final_df['Pattern'] = final_df['id'].str.split('_', n=1).str[1]
#final_df['Pattern'] = final_df['Pattern'].str.replace('_', '.', regex=False)
final_df['series_no'] = final_df['id'].str.split('_', n=2).str[2]
final_df['Groupwise_Rank'] = final_df['series_no'].rank(method='dense', ascending=True).astype(int)
final_df['Tournment_serial'] = final_df['name'].str.split('_', n=1).str[1]
# Concatenate the columns and declare the result as a float
final_df['concatenate'] = final_df['Tournment_serial'].astype(str) + '.' + final_df['series_no'].astype(str)

# Convert the concatenated string to float
#final_df['concatenate'] = final_df['concatenate'].astype(float)

# Rank the 'Pattern' column in ascending order and create a new column 'Rank'
final_df['Rank'] = final_df['Pattern'].rank(method='dense').astype(int)

final_df.head(5)


,name,start_date,end_date,tours,time_control,white,black,date,result,id,Pattern,series_no,Groupwise_Rank,Tournment_serial,concatenate,Rank
0,tournament_1,2014-01-08,2014-01-17,9,rapid,贾叶珍,范辰妮,2014-01-08,0.5,tournament_1_1,1_1,1,1,1,1.1,1497
1,tournament_1,2014-01-08,2014-01-17,9,rapid,吕亚光,李嘉爵,2014-01-08,0.5,tournament_1_2,1_2,2,112,1,1.2,1608
2,tournament_1,2014-01-08,2014-01-17,9,rapid,刘奇喜,刘晓鹏,2014-01-08,0.5,tournament_1_3,1_3,3,223,1,1.3,1717
3,tournament_1,2014-01-08,2014-01-17,9,rapid,陆桂姐,郑新聪,2014-01-08,1.0,tournament_1_4,1_4,4,334,1,1.4,1728
4,tournament_1,2014-01-08,2014-01-17,9,rapid,李汶玲,叶天英,2014-01-08,0.5,tournament_1_5,1_5,5,378,1,1.5,1739


In [146]:
import pandas as pd

# Create a combined column to represent priority
# Here, we prioritize 'Tournment_serial' and then 'series_no'
final_df['combined1'] = final_df['Tournment_serial'].astype(str) + '_' + final_df['series_no'].astype(str)

# Rank the combined column. Use `rank()` to get a ranking based on priority.
final_df['series_Rank1'] = final_df['combined1'].rank(method='dense').astype(int)

# Drop the combined column if no longer needed
#final_df.drop(columns=['combined1'], inplace=True)

# Display the updated DataFrame
print(final_df)


                name  start_date    end_date  tours time_control white black  \
0       tournament_1  2014-01-08  2014-01-17      9        rapid   贾叶珍   范辰妮   
1       tournament_1  2014-01-08  2014-01-17      9        rapid   吕亚光   李嘉爵   
2       tournament_1  2014-01-08  2014-01-17      9        rapid   刘奇喜   刘晓鹏   
3       tournament_1  2014-01-08  2014-01-17      9        rapid   陆桂姐   郑新聪   
4       tournament_1  2014-01-08  2014-01-17      9        rapid   李汶玲   叶天英   
...              ...         ...         ...    ...          ...   ...   ...   
3257  tournament_103  2014-09-08  2014-09-08     12      classic   杨燕波    李海   
3258  tournament_103  2014-09-08  2014-09-08     12      classic   秦常明   沈健琴   
3259  tournament_103  2014-09-08  2014-09-08     12      classic    张跃   李苏瑾   
3260  tournament_103  2014-09-08  2014-09-08     12      classic   程浩婷   杨俊瑞   
3261  tournament_103  2014-09-08  2014-09-08     12      classic   秦守云    张伟   

            date  result               

In [183]:
import pandas as pd

# Assuming final_df is already created and contains data

# Define the file path where you want to save the CSV file
file_path = r'F:\jupyter\final_df4.csv'

# Save the DataFrame to a CSV file
final_df.to_csv(file_path, index=False)

print(f"DataFrame has been saved to {file_path}")


DataFrame has been saved to F:\jupyter\final_df4.csv


In [169]:
import pandas as pd

# Sample DataFrame
final_df5 = pd.DataFrame({
    'series_no': [30, 10, 20, 20, 10]
})

# Rank the 'series_no' column in ascending order using dense ranking
final_df5['Rank'] = final_df5['series_no'].rank(method='dense', ascending=True).astype(int)
final_df['Groupwise_Rank'] = final_df['series_no'].rank(method='dense', ascending=True).astype(int)

# Sort the DataFrame by 'series_no' in ascending order
final_df_sorted = final_df5.sort_values(by='series_no').reset_index(drop=True)

# Display the updated DataFrame
print(final_df_sorted)


   series_no  Rank
0         10     1
1         10     1
2         20     2
3         20     2
4         30     3


In [207]:
import pandas as pd

# Assuming final_df already exists
# Copy the DataFrame to final_df6
final_df6 = final_df.copy()

# Extract 'series_no' from 'id' by splitting on '_' and taking the third part
final_df6['series_no'] = final_df['id'].str.split('_', n=2).str[2]

# Perform dense ranking on the 'series_no' column in ascending order
final_df6['dense_rank'] = final_df6['series_no'].rank(method='dense', ascending=True).astype(int)

# Display the updated DataFrame
print(final_df6)


                name  start_date    end_date  tours time_control white black  \
0       tournament_1  2014-01-08  2014-01-17      9        rapid   贾叶珍   范辰妮   
1       tournament_1  2014-01-08  2014-01-17      9        rapid   吕亚光   李嘉爵   
2       tournament_1  2014-01-08  2014-01-17      9        rapid   刘奇喜   刘晓鹏   
3       tournament_1  2014-01-08  2014-01-17      9        rapid   陆桂姐   郑新聪   
4       tournament_1  2014-01-08  2014-01-17      9        rapid   李汶玲   叶天英   
...              ...         ...         ...    ...          ...   ...   ...   
3257  tournament_103  2014-09-08  2014-09-08     12      classic   杨燕波    李海   
3258  tournament_103  2014-09-08  2014-09-08     12      classic   秦常明   沈健琴   
3259  tournament_103  2014-09-08  2014-09-08     12      classic    张跃   李苏瑾   
3260  tournament_103  2014-09-08  2014-09-08     12      classic   程浩婷   杨俊瑞   
3261  tournament_103  2014-09-08  2014-09-08     12      classic   秦守云    张伟   

            date  result               

In [211]:
import pandas as pd

# Assuming final_df is already created and contains data

# Define the file path where you want to save the CSV file
file_path = r'F:\jupyter\final_df6.csv'

# Save the DataFrame to a CSV file
final_df6.to_csv(file_path, index=False)

print(f"DataFrame has been saved to {file_path}")


DataFrame has been saved to F:\jupyter\final_df6.csv
